In [9]:
from langchain_community.llms.moonshot import Moonshot
from langgraph.graph import  Graph
moonshot = Moonshot(model="moonshot-v1-8k")
from typing import Dict, Any
from LLM.data_info import data_info

In [2]:
rpn_example='D_ts_std(D_Minute_weight_mean(M_cs_umr(M_at_div(M_at_div(M_ts_mean_mid_neighbor(M_V, 3), M_ts_mean_left_neighbor(M_V, 3)), M_at_div(M_ts_delay(M_V, 2), M_ts_mean_right_neighbor(M_V, 5))), M_at_add(M_at_div(M_ts_mean_left_neighbor(M_C, 10), M_ts_mean_mid_neighbor(M_L, 2)), M_cs_scale(M_ts_mean_right_neighbor(M_C, 2)))), M_toD_standard(M_cs_rank(M_ts_mean_mid_neighbor(M_L, 2)), D_ts_norm(D_ts_delay(D_L, 20), 2))), 2)'

parser_example={
    'tree':['D_ts_std(subtree_ARG0,2)'],
    'subtree':['D_Minute_weight_mean(trunk_ARG0,trunk_ARG1)'],
    'trunk':['M_cs_umr(root_ARG0,root_ARG1)','M_toD_standard(root_ARG2,root_ARG3)'],
    'root':['M_cs_umr(M_at_div(M_at_div(seed_ARG0,seed_ARG1), M_at_div(seed_ARG2, seed_ARG3)), M_at_add(M_at_div(seed_ARG4, seed_ARG5, M_cs_scale(seed_ARG6)))',
            'M_toD_standard(M_cs_rank(seed_ARG7), seed_ARG8)'],
    'seed':['M_ts_mean_mid_neighbor(M_V, 3)',
            'M_ts_mean_left_neighbor(M_V, 3)',
            'M_ts_delay(M_V, 2)',
            'M_ts_mean_right_neighbor(M_V, 5)',
            'M_ts_mean_left_neighbor(M_C, 10)',
            'M_ts_mean_mid_neighbor(M_L, 2)',
            'M_ts_mean_right_neighbor(M_C, 2)',
            'M_ts_mean_mid_neighbor(M_L, 2)',
            'D_ts_delay(D_L, 20)',]
}

In [ ]:
root= {'abbreviation':
    ['M_cs_umr(M_at_div(M_at_div(seed_ARG0,seed_ARG1), M_at_div(seed_ARG2, seed_ARG3)), M_at_add(M_at_div(seed_ARG4, seed_ARG5, M_cs_scale(seed_ARG6)))',
    'M_toD_standard(M_cs_rank(seed_ARG7), seed_ARG8)'],
       'tree_mode':list[int],
       }

In [7]:
graph = Graph()

In [ ]:
initial_state={
    'input_text':parser_example,
    'context':{
        'project':'你要按照我下面的descripition完成针对input_text的理解',
        'sequence': ['seed','root','trunk','subtree','tree'],
        'description':'sequence里面告诉了你处理顺序，下层是上层的输入，比如root的输入是seed；你要根据下层输入的处理后的数据，结合当前层的算子，考虑当前层处理以后输出的东西有怎样具体的逻辑含义'
    },
    'current_step_output':{
    }  
}

In [ ]:
def generate_node_func(layer:str,node_id: int,prompt):
    def node_func(state: Dict[str, Any]) -> Dict[str, Any]:
        input_text = state.get("input_text", "")
        op_info = state.get("context", {}).get("op_info", "")
        data_info = state.get("context", {}).get("data_info", "")
    
        # 构建提示词
        prompt = f"""
        请你根据以下信息，考虑处理完的数据有怎样的逻辑含义：
        - 输入文本: {input_text}
        - 操作信息: {op_info}
        - 数据信息: {data_info}
        """
        response = moonshot.invoke(prompt)
        
        state[f"node_{node_id}_output"] = response
        return state
    
    return node_func

In [ ]:
def parse_input(organ:str) -> int:
    return len(parser_example[organ])

num_nodes = parse_input(parser_example)

def initial_node(state: Dict[str, Any]) -> Dict[str, Any]:
    state["initial_output"] = "Initial output"
    return state

graph.add_node("initial_node", initial_node)
